In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import tensorflow as tf
import numpy as np

### 查看tensorflow版本

In [5]:
tf.__version__

'1.4.0'

### 点积运算
1. 随机生成矩阵数据，np.random.rand，指定(行,列)
2. np.dot进行矩阵运算，正常矩阵运算

In [7]:
np.dot([1,2],[[3,4],[5,6]])

array([13, 16])

In [8]:
# 使用 NumPy 生成假数据(phony data), 总共 100 个点.
#点积运算：(1,2)*(2,100)
x_data = np.float32(np.random.rand(2, 100)) # 随机输入
y_data = np.dot([0.100, 0.200], x_data) + 0.300

### tensorflow先构建图模型
1. tf.Variable是变量
2. tf.zeros用于生成指定shape的零向量
3. tf.random_uniform用于生成指定shape的均匀分布随机变量
4. tf.matmul用于向量计算，两个参数的shape分别为(a,k)和(k,b)

In [11]:
# 构造一个线性模型
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
y = tf.matmul(W, x_data) + b

### tensorflow常用函数
1. tf.reduce_mean函数，reduce是指定维度的意思，在某维度上求均值
2. tf.square函数，计算平方值
3. tf.train.GradientDescentOptimizer函数，梯度下降算法，使用恒定学习率；自适应学习率，使用tf.train.AdagradOptimizer和tf.train.AdamOptimizer
4. tf.train.GradientDescentOptimizer(0.5).minimize(loss)用于获取最优化处理。

In [14]:
# 最小化方差
loss = tf.reduce_mean(tf.square(y - y_data))
# 声明梯度下降学习器
optimizer = tf.train.GradientDescentOptimizer(0.5)
# 优化处理
train = optimizer.minimize(loss)

### 初始化以及运行

In [15]:
# 初始化变量
init = tf.global_variables_initializer()

In [16]:
# 启动图 (graph)
sess = tf.Session()
sess.run(init)

### 10次不同初始化拟合

In [17]:
# 拟合平面
for step in range(0, 201):  #python3里没有xrange
    sess.run(train)
    if step % 20 == 0:
        print (step, sess.run(W), sess.run(b))

0 [[ 0.6414485  -0.48986286]] [0.8037029]
20 [[0.13447842 0.04432093]] [0.36207584]
40 [[0.09720384 0.16839905]] [0.31785136]
60 [[0.09707775 0.19292541]] [0.30522746]
80 [[0.09879377 0.19826144]] [0.3015463]
100 [[0.09958623 0.19953986]] [0.30045992]
120 [[0.09986776 0.19987182]] [0.30013722]
140 [[0.09995906 0.19996312]] [0.30004102]
160 [[0.09998753 0.1999892 ]] [0.30001226]
180 [[0.09999621 0.1999968 ]] [0.30000368]
200 [[0.09999885 0.19999905]] [0.3000011]


TensorFlow Python API 依赖 Python 2.7 版本。

Docker 是一个开源的应用容器引擎，让开发者可以打包他们的应用以及依赖包到一个可移植的容器中,然后发布到任何流行的Linux机器或Windows 机器上,也可以实现虚拟化,容器是完全使用沙箱机制,相互之间不会有任何接口。

### 常量计算
1. tf.constant用于常量

In [18]:
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


In [19]:
a = tf.constant(10)
b = tf.constant(32)
print(sess.run(a+b))

42


### TensorFlow计算逻辑
1. 使用图 (graph) 来表示计算任务.
2. 在被称之为 会话 (Session) 的上下文 (context) 中执行图.
3. 使用 tensor 表示数据.
4. 通过 变量 (Variable) 维护状态.
5. 使用 feed 和 fetch 可以为任意的操作(arbitrary operation) 赋值或者从其中获取数据.

TensorFlow 是一个编程系统, 使用图来表示计算任务. 图中的节点被称之为 op。
1. tf.matmul用于矩阵计算

In [20]:
# 创建一个常量 op, 产生一个 1x2 矩阵. 这个 op 被作为一个节点
# 加到默认图中.
#
# 构造器的返回值代表该常量 op 的返回值.
matrix1 = tf.constant([[3., 3.]])

# 创建另外一个常量 op, 产生一个 2x1 矩阵.
matrix2 = tf.constant([[2.],[2.]])

# 创建一个矩阵乘法 matmul op , 把 'matrix1' 和 'matrix2' 作为输入.
# 返回值 'product' 代表矩阵乘法的结果.
product = tf.matmul(matrix1, matrix2)

In [21]:
# 启动默认图.
sess = tf.Session()

# 调用 sess 的 'run()' 方法来执行矩阵乘法 op, 传入 'product' 作为该方法的参数. 
# 上面提到, 'product' 代表了矩阵乘法 op 的输出, 传入它是向方法表明, 我们希望取回
# 矩阵乘法 op 的输出.
#
# 整个执行过程是自动化的, 会话负责传递 op 所需的全部输入. op 通常是并发执行的.
# 
# 函数调用 'run(product)' 触发了图中三个 op (两个常量 op 和一个矩阵乘法 op) 的执行.
#
# 返回值 'result' 是一个 numpy `ndarray` 对象.
result = sess.run(product)
print (result)

sess.close()

[[12.]]


### 执行时，可指定gpu

In [14]:
#指定gpu, cpu
with tf.Session() as sess:
    with tf.device("/gpu:1"):
        matrix1 = tf.constant([[3., 3.]])
        matrix2 = tf.constant([[2.],[2.]])
        product = tf.matmul(matrix1, matrix2)
        ...

目前支持的设备包括:
1. "/cpu:0": 机器的 CPU.
2. "/gpu:0": 机器的第一个 GPU, 如果有的话.
3. "/gpu:1": 机器的第二个 GPU, 以此类推.

### 交互式

In [22]:
# 进入一个交互式 TensorFlow 会话.
import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# 使用初始化器 initializer op 的 run() 方法初始化 'x' 
x.initializer.run()

# 增加一个减法 sub op, 从 'x' 减去 'a'. 运行减法 op, 输出结果 
sub = tf.subtract(x, a)  #python3里没有sub
print(sub.eval())

[-2. -1.]


可以把 TensorFlow tensor 看作是一个 n 维的数组或列表. 一个 tensor 包含一个静态类型 rank, 和 一个 shape. 

## 计数器实现
1. 创建一个变量tf.Variable，初始化为0
2. 创建一个常量tf.constant，初始化为1
3. 定义加1动作
4. 定义赋值动作
5. 初始化，循环执行加1并赋值动作

In [24]:
# 创建一个变量, 初始化为标量 0.
state = tf.Variable(0, name="counter")

# 创建一个 op, 其作用是使 state 增加 1
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# 启动图后, 变量必须先经过`初始化` (init) op 初始化,
# 首先必须增加一个`初始化` op 到图中.
init_op = tf.global_variables_initializer()

# 启动图, 运行 op
with tf.Session() as sess:
  # 运行 'init' op
  sess.run(init_op)
  # 打印 'state' 的初始值
  print(sess.run(state))
  # 运行 op, 更新 'state', 并打印 'state'
  for _ in range(3):
    sess.run(update)
    print (sess.run(state))

0
1
2
3


## Fetch
取回操作的输出内容

In [25]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
    result = sess.run([mul, intermed])
    print (result)

[21.0, 7.0]


## Feed
临时替代图中的任意操作中的 tensor 可以对图中任何操作提交补丁, 直接插入一个 tensor.

In [26]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))

[array([14.], dtype=float32)]
